In [1]:
cd ..

/home/soda/rcappuzz/work/benchmark-join-suggestions


In [29]:
import pickle
from pathlib import Path
import polars as pl

In [38]:
tgt_files = [
    "generated_candidates_company-employees-prepared.pickle",
    "generated_candidates_us-accidents-prepared-new.pickle",
    "generated_candidates_presidential-results-prepared-new.pickle",
]

In [39]:
ff = tgt_files[1]

In [40]:
for ff in tgt_files:
    candidates_index = pickle.load(open(ff, "rb"))
    candidates = candidates_index["minhash"]
    list_dict=[]
    print(ff, len(candidates))
    for c, cand in candidates.items():
        src_md = cand.source_metadata
        cand_md = cand.candidate_metadata
        left_on = cand.left_on
        right_on = cand.right_on
        source_table = pl.read_parquet(src_md["full_path"])
        candidate_table = pl.read_parquet(cand_md["full_path"])
        # containment=jp.measure_containment(source_table, candidate_table, left_on, right_on)
        merged = source_table.join(
            candidate_table,
            left_on=left_on,
            right_on=right_on,
            how="left"
        )
        pre = len(source_table)
        post = len(merged)
        c_dict = {
            "source_table": src_md["full_path"],
            "candidate_table": cand_md["full_path"],
            "similarity": cand.similarity_score,
            "size_prejoin": pre,
            "size_postjoin": post,
            "scale_factor": post/pre
        }
        list_dict.append(c_dict)
    df_res = pl.from_dicts(list_dict)

    print(df_res.select(
        pl.col("source_table").apply(lambda x: Path(x).stem),
        pl.col("scale_factor").mean().alias("sf_mean"),
        pl.col("scale_factor").max().alias("sf_max"),
    )[0])

generated_candidates_company-employees-prepared.pickle 243


KeyboardInterrupt: 

In [43]:
for ff in tgt_files:
    candidates_index = pickle.load(open(ff, "rb"))
    candidates = candidates_index["minhash"]
    list_dict=[]
    print(ff, len(candidates))


generated_candidates_company-employees-prepared.pickle 243
generated_candidates_us-accidents-prepared-new.pickle 10
generated_candidates_presidential-results-prepared-new.pickle 130


In [37]:
candidates

{'1d721d09aa48f01cc669b5ff34b84b05': <src.utils.data_structures.CandidateJoin at 0x7eff2e360430>,
 'bc86268530cb8edbd92d2711ad985c77': <src.utils.data_structures.CandidateJoin at 0x7efeb9974430>}